
## Licence
Copyright (c) 2022, RTE (http://www.rte-france.com)
This Source Code Form is subject to the terms of the Mozilla Public
License, v. 2.0. If a copy of the MPL was not distributed with this
file, You can obtain one at http://mozilla.org/MPL/2.0/.


## Author
Hugo Schindler <hugo.schindler at rte-france.com>


## Description
This notebook illustrates the flow decomposition algorithm results.
All plots are in MW (if not explicitly normalized).
A toy network is provided. Do not hesitate to load your network !

If you have issues with the load flow provider, set the following lines in your ~/.itools/config.yml  
```
load-flow:
 default-impl-name: OpenLoadFlow
```

In [1]:
import random
import numpy as np
import pandas as pd

import plotly.express as px

import pycountry

import pypowsybl as pp

pd.options.display.max_columns = None
pd.options.display.expand_frame_repr = False


In [2]:
colors = px.colors.qualitative.Dark24 + px.colors.qualitative.Light24
random.Random(42).shuffle(colors)

# Load a network

You can load your own network with 
net = pp.network.load(...)

In [3]:
def assign_country_all_substations(net):
    substation_ids = net.get_substations().index
    all_countries = ["AT", "BE", "BG", "CY", "CZ", "DE", "DK", "EE", "ES", "FI", "FR", "GR", "HR", "HU", "IE", "IT", "LT", "LU", "LV", "NL", "PL", "PT", "RO", "SE", "SI", "SK"]
    number_country = 20
    random.Random(42).shuffle(all_countries)
    substation_countries = random.Random(42).choices(random.Random(42).choices(all_countries, k=number_country), k=len(substation_ids))
    return dict(zip(substation_ids, substation_countries))

def propagate_country_nearby_substations(net, dss):
    vl = net.get_voltage_levels()
    l = net.get_lines()
    df_net = pd.merge(
        pd.merge(l, vl.add_suffix("_vl1"), right_index=True, left_on="voltage_level1_id"),
        vl.add_suffix("_vl2"), right_index=True, left_on="voltage_level2_id")
    connected_ss = df_net[["substation_id_vl1", "substation_id_vl2"]].values
    for _ in range(3):
        for connected_sub in connected_ss:
            dss[connected_sub[1]] = dss[connected_sub[0]]
    net.update_substations(id=list(dss.keys()), country=list(dss.values()))

def get_upgraded_ieee_net():
    net = pp.network.create_ieee300()

    # add country to substations
    dss = assign_country_all_substations(net)
    propagate_country_nearby_substations(net, dss)

    # generator fix: otherwise they are discarded because of not plausible Pmax
    gen = net.get_generators().index
    net.update_generators(id=gen, max_p=[2000]*len(gen), min_p=[-2000]*len(gen))

    #net.dump("/tmp/test-net.xiidm")

    return net

In [4]:
net = get_upgraded_ieee_net()

# Run a flow decomposition

Running a flow decomposition with those parameter might take a while and a lot of RAM.
The default parameters consume less ressources.

We also add a few useful columns to the dataframe:
- total loop flow
- total flow

All the computation are donc in state N.

In [5]:
parameters = pp.flowdecomposition.Parameters(enable_losses_compensation=True,
    rescale_enabled=False,
    xnec_selection_strategy=pp.flowdecomposition.XnecSelectionStrategy.INTERCONNECTION_OR_ZONE_TO_ZONE_PTDF_GT_5PC,
    contingency_strategy=pp.flowdecomposition.ContingencyStrategy.ONLY_N_STATE,
    )
flow_dec_original=pp.flowdecomposition.run(net, flow_decomposition_parameters=parameters)
flow_dec_original

,branch_id,contingency_id,country1,country2,ac_reference_flow,dc_reference_flow,commercial_flow,internal_flow,loop_flow_from_bg,loop_flow_from_cz,loop_flow_from_dk,loop_flow_from_ee,loop_flow_from_fr,loop_flow_from_hu,loop_flow_from_ie,loop_flow_from_lt,loop_flow_from_lu,pst_flow
xnec_id,,,,,,,,,,,,,,,,,,
L105-110-1_InitialState,L105-110-1,InitialState,LT,LU,41.955416,41.955416,21.886904,0.000000,1.283926e-01,0.617328,-7.145688e-01,3.554652e-01,-7.566125e+00,4.588604e-01,0.0,3.716386e+00,2.375047e+01,0.0
L109-114-1_InitialState,L109-114-1,InitialState,LU,BG,6.371956,6.371956,31.792257,0.000000,-1.201865e+00,0.649216,-8.347491e-01,9.320963e-01,-1.282324e+01,3.609872e-01,0.0,-4.697886e+00,-5.985919e+00,0.0
L112-114-1_InitialState,L112-114-1,InitialState,LU,BG,-8.582649,-8.582649,-37.758978,0.000000,6.666285e-01,-0.965217,1.359790e+00,-8.581408e-01,1.494273e+01,-9.058846e-01,0.0,4.152100e+00,2.628695e+01,0.0
L115-122-1_InitialState,L115-122-1,InitialState,DK,HU,48.481203,48.481203,241.016015,0.000000,6.239453e-14,0.247209,-1.040830e+02,-1.006140e-15,-2.486900e-14,-8.869903e+01,0.0,1.190714e-13,3.874678e-14,0.0
L116-120-1_InitialState,L116-120-1,InitialState,HU,HU,-60.950896,-60.950896,249.321866,-260.591751,-3.219647e-15,-0.047271,7.226805e+01,-1.202163e-15,-3.119727e-14,0.000000e+00,0.0,5.018208e-14,1.625089e-14,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T7139-139-1_InitialState,T7139-139-1,InitialState,DK,DK,700.736825,700.736825,382.462905,317.512015,7.549517e-15,0.761905,0.000000e+00,4.357625e-15,1.101341e-13,-3.774758e-14,0.0,2.562395e-13,-4.218847e-14,0.0
T7166-166-1_InitialState,T7166-166-1,InitialState,LT,LT,553.736825,553.736825,91.248074,461.726847,1.199041e-14,0.761905,-2.877698e-13,-1.182388e-14,1.314504e-13,-5.662137e-14,0.0,0.000000e+00,-1.332268e-15,0.0
T73-74-1_InitialState,T73-74-1,InitialState,BG,BG,-253.282574,-253.282574,193.043741,56.477814,0.000000e+00,17.681877,1.133176e+00,2.128027e+00,-4.166597e+01,-1.888317e+00,0.0,2.890972e+01,-2.976571e-01,0.0


In [6]:
flow_dec = flow_dec_original.copy()
set(flow_dec['contingency_id'])

{'InitialState'}

In [7]:
flow_dec = flow_dec[flow_dec['contingency_id'] == 'InitialState']
flow_dec['total_flow'] = flow_dec[[c for c in flow_dec.columns if ('reference' not in c and 'flow' in c)]].sum(axis=1)
flow_dec['total_loop_flow'] = flow_dec[[c for c in flow_dec.columns if 'loop_flow_from_' in c]].sum(axis=1)

def alpha_2_to_country(l):
    return [pycountry.countries.get(alpha_2=alpha_2) for alpha_2 in l]

countries_alpha2 = set(flow_dec["country1"]).union(set(flow_dec["country2"]))
countries = alpha_2_to_country(countries_alpha2)
countries

[Country(alpha_2='IE', alpha_3='IRL', flag='🇮🇪', name='Ireland', numeric='372'),
 Country(alpha_2='LT', alpha_3='LTU', flag='🇱🇹', name='Lithuania', numeric='440', official_name='Republic of Lithuania'),
 Country(alpha_2='CZ', alpha_3='CZE', flag='🇨🇿', name='Czechia', numeric='203', official_name='Czech Republic'),
 Country(alpha_2='DK', alpha_3='DNK', flag='🇩🇰', name='Denmark', numeric='208', official_name='Kingdom of Denmark'),
 Country(alpha_2='BG', alpha_3='BGR', flag='🇧🇬', name='Bulgaria', numeric='100', official_name='Republic of Bulgaria'),
 Country(alpha_2='LU', alpha_3='LUX', flag='🇱🇺', name='Luxembourg', numeric='442', official_name='Grand Duchy of Luxembourg'),
 Country(alpha_2='HU', alpha_3='HUN', flag='🇭🇺', name='Hungary', numeric='348', official_name='Hungary'),
 Country(alpha_2='EE', alpha_3='EST', flag='🇪🇪', name='Estonia', numeric='233', official_name='Republic of Estonia'),
 Country(alpha_2='FR', alpha_3='FRA', flag='🇫🇷', name='France', numeric='250', official_name='Fr

# Flow decomposition top bar charts

The top lines are selected given a metric.

In [8]:
def flow_decomposition_bar_chart(sorting_column, ascending=False, head=20, plot_scatter=True):
    threshold = .05
    df = flow_dec.sort_values(sorting_column, ascending = ascending).head(head).copy()
    df_p = df[[c for c in flow_dec.columns if ('reference' not in c and 'total' not in c and 'flow' in c)]]
    df_m = df_p.abs().div(df_p.abs().sum(axis=1), axis=0) > threshold
    df_f = df_p[df_m]
    df_f['masked_flows_positive'] = df_p[df_p[df_m == False]>0].sum(axis=1)
    df_f['masked_flows_negative'] = df_p[df_p[df_m == False]<0].sum(axis=1)
    fig = px.bar(df_f,
        orientation='h',
        color_discrete_sequence=colors[:df_f.columns.size],
        text_auto='.0f',
        title=f'Sorted by: {sorting_column}, ascending: {ascending}, head: {head}',
        height=1000,
        labels={
            'branch_id':'Branch id',
            'value': 'Flow decomposition value',
            'variable': 'Decomposition part:',
        },
        template="seaborn",
        )
    if plot_scatter:
        fig.add_scatter(
            y=df.index,
            x=df['total_flow'],
            mode='lines+markers',
            name='total_flow'
            )
        fig.add_scatter(
            y=df.index,
            x=df['total_loop_flow'],
            mode='lines+markers',
            name='total_loop_flow'
            )
    fig.show()

In [9]:
flow_decomposition_bar_chart('total_loop_flow')

In [10]:
flow_decomposition_bar_chart('total_loop_flow', ascending=True)

In [11]:
flow_decomposition_bar_chart('total_flow')

In [12]:
flow_decomposition_bar_chart('pst_flow')

In [13]:
flow_decomposition_bar_chart('pst_flow', ascending=True)

In [14]:
flow_decomposition_bar_chart(f'loop_flow_from_{random.Random(42).choice(list(countries)).alpha_2.lower()}')

In [15]:
flow_decomposition_bar_chart(f'loop_flow_from_{random.Random(42).choice(list(countries)).alpha_2.lower()}', ascending=True)

In [16]:
flow_decomposition_bar_chart(f'loop_flow_from_{random.Random(12).choice(list(countries)).alpha_2.lower()}')

# Loop flow repartition from source

Loop flows are sum for each source.

In [17]:
c1, c2 = 'country1', 'country2'
df_sum1 = flow_dec.groupby(c1).sum().transpose()
df_sum2 = flow_dec.groupby(c2).sum().transpose()
df_lf_per_country = pd.concat([df_sum1, df_sum2]).groupby(level=0).sum()/2

In [18]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]].transpose()
df_s = df_p.abs().sum(axis=1)
df_p = df_p.reindex(df_s.sort_values(ascending=False).index)
df_m = df_p.abs().div(df_p.abs().sum(axis=1), axis=0) > threshold
df_f = df_p[df_m]
df_f['masked_positive'] = df_p[df_p[df_m == False]>0].sum(axis=1)
df_f['masked_negative'] = df_p[df_p[df_m == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.0f',
    title='',
    height=1000,
    labels={
        'index':'Origin of loop flow',
        'value': 'Loop flow value per origin',
        'variable': 'Impacted country',
    }
    )
fig.show()

In [19]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]].transpose()
df_m = df_p.div(df_p.abs().sum(axis=1), axis=0)
df_s = df_m[df_m < 0].sum(axis=1)
df_m = df_m.reindex(df_s.sort_values(ascending=False).index)
df_n = df_m.abs() > threshold
df_f = df_m[df_n]
df_f['masked_positive'] = df_m[df_p[df_n == False]>0].sum(axis=1)
df_f['masked_negative'] = df_m[df_p[df_n == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.2f',
    height=1000,
    labels={
        'index':'Origin of loop flow',
        'value': 'Loop flow value normalized per origin',
        'variable': 'Impacted country',
    }
    )
fig.show()

# Loop flow repartition for visited countries

Loop flow are sum for visited countries

In [20]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]]
df_s = df_p.abs().sum(axis=1)
df_p = df_p.reindex(df_s.sort_values(ascending=False).index)
df_m = df_p.abs().div(df_p.abs().sum(axis=1), axis=0) > threshold
df_f = df_p[df_m]
df_f['masked_positive'] = df_p[df_p[df_m == False]>0].sum(axis=1)
df_f['masked_negative'] = df_p[df_p[df_m == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.0f',
    title='',
    height=1000,
    labels={
        'index':'Destination of loop flow',
        'value': 'Loop flow value per origin',
        'variable': 'Origin country',
    }
    )
fig.show()

In [21]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]]
df_m = df_p.div(df_p.abs().sum(axis=1), axis=0)
df_s = df_m[df_m < 0].sum(axis=1)
df_m = df_m.reindex(df_s.sort_values(ascending=False).index)
df_n = df_m.abs() > threshold
df_f = df_m[df_n]
df_f['masked_positive'] = df_m[df_p[df_n == False]>0].sum(axis=1)
df_f['masked_negative'] = df_m[df_p[df_n == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.2f',
    height=1000,
    labels={
        'index':'Destination of loop flow',
        'value': 'Loop flow value normalized per origin',
        'variable': 'Origin country',
    }
    )
fig.show()

# Loop flow heat map

Provide a view

In [22]:
df_matrix = df_lf_per_country.transpose().sort_index(axis=1).sort_index()

df_matrix = df_matrix[[c for c in df_matrix.columns if ('loop_flow_from' in c)]].transpose()
df_matrix.index = df_matrix.index.map(lambda c: c.split("_")[-1].upper())
df_matrix.index = [country.name for country in alpha_2_to_country(df_matrix.index)]
df_matrix.columns = [country.name for country in alpha_2_to_country(df_matrix.columns)]
fig = px.imshow(np.sign(df_matrix)*np.log10(df_matrix.abs()+1),
    labels=dict(x="Visited country", y="Source country", color="Loop flow"),
    color_continuous_midpoint=0.0,
    color_continuous_scale="RdBu_r",
    height=800)
fig.update(data=[{'customdata': df_matrix,
    'hovertemplate': 'Source country: %{y}<br>Visited country: %{x}<br>Loop flow: %{customdata:.2f}<extra></extra>'}])
int_log_abs_max = int(np.log10(max(df_matrix.max().max(), abs(df_matrix.min().min()))))
tickvals = np.arange(-int_log_abs_max, int_log_abs_max+1)
fig.update_layout(coloraxis_colorbar=dict(
    #title="Population",
    tickvals=tickvals,
    ticktext=[np.sign(v)*10**abs(v) for v in tickvals],
))
fig.show()

df_matrix rows are sources, columns are destinations

# Choropleth

In [23]:
def plot_map(source = None, visited = None):
    if (visited is None) == (source is None) :
        raise Exception("Exactly one of visited or source should be None.")
    if visited is None:
        df = pd.DataFrame(df_matrix.loc[source, :])
        country = source
        directionSelected = "source"
    else:
        df = pd.DataFrame(df_matrix.loc[:, visited])
        country = visited
        directionSelected = "visited"

    df = df.rename(columns={country:'Loop Flow'})
    source_country_col_name = 'Source country'
    visited_country_col_name = 'Visited country'
    df[source_country_col_name] = df.index if source is None else [country] * len(df.index)
    df[visited_country_col_name] = df.index if visited is None else [country] * len(df.index)
    df['iso_alpha'] = [pycountry.countries.get(name=c).alpha_3 for c in df.index]
    df['log_loop_flow'] = np.sign(df['Loop Flow'])*np.log10(df['Loop Flow'].abs()+1)

    fig = px.choropleth(df, 
                        locations="iso_alpha",
                        color="log_loop_flow",
                        hover_data={source_country_col_name: True, visited_country_col_name: True, 'Loop Flow':':.1f', 'log_loop_flow':False, 'iso_alpha':False},
                        color_continuous_midpoint=0.0,
                        color_continuous_scale="RdBu_r",
                        width=1200,
                        height=800,
                        title=f'Loop flow decomposition with {directionSelected} country = {country}',
                        )
    int_log_abs_max = int(max(df['log_loop_flow'].max(), df['log_loop_flow'].min()))
    tickvals = np.arange(-int_log_abs_max, int_log_abs_max+1)
    fig.update_layout(coloraxis_colorbar=dict(
        title="Loop Flow",
        tickvals=tickvals,
        ticktext=[np.sign(v)*10**abs(v) for v in tickvals],
    ))
    fig.update_geos(
        lataxis_range=[30, 70],
        lonaxis_range=[-20, 50]
    )
    fig.show()

In [24]:
country = random.Random(12).choice(list(countries)).name
#country = pycountry.countries.get(alpha_2='CH').name # "France"
plot_map(source=country)

In [25]:
plot_map(visited=country)